In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras
import os
import zipfile

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
zip_filepath = "/content/drive/MyDrive/klimb_llm_optimization_challenge.zip"

In [4]:
extract_path = "/content/drive/MyDrive"

In [5]:
train_file_to_extract = "klimb_llm_optimization_challenge/seg_train"

In [6]:
test_file_to_extract = "klimb_llm_optimization_challenge/seg_test/"

In [7]:
os.path.isfile(zip_filepath)

True

In [8]:
with zipfile.ZipFile(zip_filepath, "r") as zip_ref:
  if not zip_ref.namelist():
    print("No items found in archive")
  else:
    for item in zip_ref.namelist():
      zip_ref.extract(item, extract_path)


KeyboardInterrupt: 

In [9]:
# Path of train and test data
train_dir = "/content/drive/MyDrive/klimb_llm_optimization_challenge/seg_train/"
test_dir = "/content/drive/MyDrive/klimb_llm_optimization_challenge/seg_test/"

In [10]:
# Data configs
batch_size = 32
img_height = 150
img_width = 150
seed = 123

In [11]:
# Load train data
train_ds, validation_ds = tf.keras.utils.image_dataset_from_directory(
train_dir,
  validation_split=0.2,
  subset="both",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size,)

Using 11228 files for training.
Using 2806 files for validation.


In [12]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [13]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3000 files belonging to 6 classes.


In [14]:
# Training the MASTER Model - using Transfer Learning
# Here we are using ImageNet pre-trained model weights
base_model = keras.applications.ResNet152(
		weights='imagenet',  # Load weights pre-trained on ImageNet.
		input_shape=(img_height, img_width, 3),
		include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False


234698864/234698864 [==============================] - 11s 0us/step


In [15]:
inputs = keras.Input(shape=(img_height, img_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
teacher_model = keras.Model(inputs, outputs)

In [16]:
teacher_model.summary()
teacher_model.compile(
		optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001),
		loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=[keras.metrics.SparseCategoricalAccuracy()])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 5, 5, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 12294     
                                                                 
Total params: 58383238 (222.71 MB)
Trainable params: 12294 (48.02 KB)
Non-trainable params: 58370944 (222.67 MB)
_________________________________________________________________


In [ ]:
epochs = 10
teacher_model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Epoch 1/10
 71/351 [=====>........................] - ETA: 43:53 - loss: 0.6577 - sparse_categorical_accuracy: 0.7685

In [ ]:
# Generate results on test data
results = teacher_model.evaluate(test_ds)
print(f"Test accuracy with trained teacher model:{results[1]*100 :.2f} %")

In [ ]:
from tensorflow.nn import softmax

In [ ]:
soft_labels_train = softmax(teacher_model.predict(train_ds), axis=-1)

In [ ]:
soft_labels_train.shape

In [ ]:
soft_labels_val = softmax(teacher_model.predict(validation_ds), axis=-1)

In [ ]:
def create_student_model(input_shape, num_classes):
  model = keras.models.Sequential()

  model.add(layers.Conv2DTranspose(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, strides=(1,1)))
  model.add(layers.MaxPooling2D((2, 2), padding='valid', strides=(1,1)))
  model.add(layers.Conv2D(64, kernel_size= (5, 5), activation='relu', strides=(1,1)))
  model.add(layers.MaxPooling2D((4,4), padding='valid', strides=(1,1)))
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dense(6))
  return model


In [ ]:
student_model = create_student_model((img_height, img_width, 3), 6)

In [ ]:
student_model.summary()

In [ ]:
sparse_categorical_accuracy = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
def train_student_model(teacher_model, student_model, train_loader, soft_labels, num_epochs=50, learning_rate=0.001):
  optimizer=keras.optimizers.legacy.Adam(learning_rate=learning_rate)
  loss_fn =keras.losses.KLDivergence()
  epoch_loss = np.array([])
  epoch_accuracy = np.array([])
  for epoch in range(num_epochs):
    batch_loss = np.array([])
    batch_accuracy = np.array([])
    for inputs, _ in train_loader:
      with tf.GradientTape() as tape:

        student_predictions = student_model(inputs, training=True)
        loss = loss_fn(softmax(teacher_model(inputs, training=False), axis=-1), softmax(student_predictions, axis=-1))
        accuracy = sparse_categorical_accuracy.update_state(y_true=softmax(teacher_model(inputs, training=False), axis=-1).numpy(), y_pred=softmax(student_predictions, axis=-1).numpy())
        batch_loss.append(loss.numpy())
        batch_accuracy.append(accuracy.numpy())

      gradients = tape.gradient(loss, student_model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))
    epoch_accuracy.append(batch_accuracy.mean())
    epoch_loss.append(batch_loss.mean())
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss[epoch]}')
    print(f"Epoch [{epoch + 1}/{num_epochs}], Accuracy: {epoch_accuracy[epoch]}")

In [ ]:
train_student_model(teacher_model, student_model, train_ds, soft_labels_train)